#Ambiente

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Datasets

## AmericanasBR

In [ ]:
#baixando os datasets
!curl https://www.inf.ufrgs.br/~viviane/DS/B2W-Reviews01_binario5000_TRAIN.csv > B2W-Reviews01_binario5000_TRAIN.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1657k  100 1657k    0     0   735k      0  0:00:02  0:00:02 --:--:--  735k


In [ ]:
df_train = pd.read_csv('B2W-Reviews01_binario5000_TRAIN.csv')

In [ ]:
classes = df_train.label.unique()
classes

array([0, 1])

# Entendendo as Embeddigns

A bilbioteca [Gensim](https://radimrehurek.com/gensim/models/word2vec.html) permite treinar e usar word embedings.

A versão da biblioteca a ser usada neste notebook é a 4.

Veja diferenças entre versão 3 e 4 neste [link](https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).

In [ ]:
!pip install gensim==4.3.1
!pip install scipy==1.10.1
# !pip install --upgrade gensim

In [ ]:
from gensim import utils
import gensim.models

In [ ]:
gensim.__version__

'4.3.1'

## Treinar embeddings

A biblioteca Gensim permite que você treine as embeddings do seu corpus

https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
# classe para montar o dataset
class PreProcess:
    def __init__(self, docs):
            self.lista_text = docs
    def __iter__(self):
        for line in self.lista_text:
            # assume there's one document per line, tokens separated by whitespace:
            yield utils.simple_preprocess(line) # este método tokeniza e faz algum preprocessamento
            # https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

Alguns parametros do [Word2vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)

*   `vector_size` – dimensionalidade dos vetores das palavras.
*   `window` – tamanho do contexto a considerar, por exemplo windows=5 irá considerar as 5 palavras à esquerda e as 5 palavras à direita da palavra atual como a janela de contexto. O modelo tentará então prever a palavra atual dado este contexto.
*   `min_count` – ignora palavras com frequência total menor do que min_count.
*   `sg` – o algoritmo de treinamento: 1 for skip-gram e diferente disto CBOW.





In [ ]:
sentences = PreProcess(df_train['text'].values)
# assim treina o modelo usando as configurações padrão e estas especificadas aqui
model = gensim.models.Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, epochs=20, sg=1)

Quando não se necessita mais do estado completo do modelo treinado (não precisa continuar treinando), `Gensim` permite separar os vetores treinados em [`KeyedVectors`](https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors) possibilitando salvar apenas os vetores e suas chaves (as palavras).



In [ ]:
# desta forma acessa somente as palavras e seus vetores
word_vectors = model.wv
word_vectors

In [ ]:
# total de palavras e as 10 primeiras
words = list(word_vectors.key_to_index)
print(f'O vocabulario contém {len(words)} palavras')
print(words[0:10])

O vocabulario contém 13905 palavras
['de', 'não', 'produto', 'que', 'muito', 'com', 'do', 'um', 'para', 'da']


In [ ]:
# verificando o id de uma palavra:
print('id de entrega:', word_vectors.key_to_index['entrega'])
print('palavra do id 4:', word_vectors.index_to_key[4])

id de entrega: 12
palavra do id 4: muito


In [ ]:
# ocorrências de uma palavra:
palavra = 'entrega'
palavra_cnt = word_vectors.get_vecattr(palavra, "count")
print(f'A palavra {palavra} ocorre {palavra_cnt} vezes no dataset')

A palavra entrega ocorre 1770 vezes no dataset


Cada palavra única do corpus é representada por um vetor de tamanho `vector_size`, que corresponde ao número de dimnesões usado no treinamento.

In [ ]:
print(f"Embeddings da palavra produto com dimensão {word_vectors['produto'].shape}")
word_vectors['produto']

Embeddings da palavra produto com dimensão (100,)


array([ 0.02128273,  0.04053448, -0.06614898, -0.00988322,  0.01017751,
       -0.45436728,  0.54336053,  0.797465  , -0.49118707, -0.37816927,
       -0.23629418, -0.2198794 ,  0.1952771 ,  0.1553641 ,  0.444322  ,
       -0.07136092,  0.40096724,  0.08944459, -0.47598237, -0.5518115 ,
        0.22160318,  0.09652169,  0.39556143, -0.03421578,  0.27388722,
       -0.24135475,  0.08464433,  0.00403222, -0.31911814,  0.22733594,
        0.20293224,  0.0072099 ,  0.02542805, -0.45521608, -0.00192987,
        0.3339316 ,  0.32709348,  0.38478574,  0.16681267,  0.18312055,
        0.07599336, -0.08060327, -0.38643715,  0.07523794,  0.08868514,
        0.22377709,  0.10779741, -0.0371132 ,  0.20903316, -0.13165003,
        0.19807391, -0.36481375, -0.11778326, -0.67842984,  0.07569831,
       -0.16853334, -0.06022951,  0.12302832, -0.03468551,  0.19314803,
       -0.05567524, -0.11593162,  0.3550867 ,  0.12177002, -0.27456105,
        0.32561556,  0.17388783,  0.02087703, -0.08935519,  0.39

In [ ]:
# possuem representações diferentes:
print(word_vectors['agua'][0:5])
print(word_vectors['água'][0:5])

[-0.32919726  0.33187085  0.35944137  0.28932017 -0.35414296]
[ 0.20658506  0.4405677   0.6668322   0.28320873 -0.1969216 ]


Salvando as embeddings treinadas:

In [ ]:
# salva o modelo em formato binario do gensim:
model.save("word2vec.model")

In [ ]:
# salva em formato texto somete as palavras e seus vetores de embeddings
word_vectors = model.wv
word_vectors.save_word2vec_format("word2vec.txt", binary= False)


## Lendo os vetores

In [ ]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('word2vec.txt', binary=False)

In [ ]:
word_vectors

In [ ]:
print('Total de palavras: ',len(word_vectors))
print('id da palavra água:', word_vectors.key_to_index['água'])
print(word_vectors['água'][0:10])

Total de palavras:  13905
id da palavra água: 206
[ 0.20658506  0.4405677   0.6668322   0.28320873 -0.1969216  -0.4837392
  0.02117063  0.6565078  -0.35674182 -0.39681745]


In [ ]:
#somente os vetores das embeddings:
vectors = word_vectors.vectors
vectors

array([[ 0.10175715,  0.08989055,  0.28448024, ..., -0.36155516,
        -0.07402344, -0.09705781],
       [-0.05960838,  0.08282295,  0.11873513, ..., -0.20836392,
        -0.18851435,  0.12739205],
       [ 0.02128273,  0.04053448, -0.06614898, ..., -0.03099371,
         0.04214196, -0.09896743],
       ...,
       [ 0.01010834,  0.13319643,  0.20950988, ..., -0.21257728,
        -0.0797138 ,  0.02427294],
       [-0.02219304,  0.2389947 ,  0.12565215, ..., -0.33679274,
        -0.09555756,  0.08789514],
       [ 0.03454965,  0.03567057,  0.05106392, ..., -0.1700989 ,
        -0.11076976, -0.00525933]], dtype=float32)

In [ ]:
# acessando o vetor da palavra água
print(vectors[206][0:10])

[ 0.20658506  0.4405677   0.6668322   0.28320873 -0.1969216  -0.4837392
  0.02117063  0.6565078  -0.35674182 -0.39681745]


## Visualizando embeddings

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

In [ ]:
%%time
tsne = TSNE(n_components=3, random_state=0)
projections = tsne.fit_transform(vectors, )

CPU times: user 11min 36s, sys: 1.04 s, total: 11min 37s
Wall time: 11min 41s


In [ ]:
dfP = pd.DataFrame(projections)
dfP['word'] = words
fig = px.scatter_3d(dfP, x=0, y=1, z=2,hover_data=['word'])
fig.update_traces(marker_size=3)
fig.show()

Também é possível usar o [Embeddings Projector](https://projector.tensorflow.org/) do Tensorflow.

## Usando embeddings já treinadas
Podemos usar word embeddings que já foram treinadas e disponibilizadas

Existem modelos disponíveis no Gensim:

In [ ]:
import gensim.downloader as api

In [ ]:
disponiveis = api.info()
disponiveis.keys()

dict_keys(['corpora', 'models'])

In [ ]:
disponiveis['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [ ]:
word_vectors = api.load("glove-wiki-gigaword-100") #128Mb

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
#é uma lista de palavras e seus vetores de embeddinsg treinados por alguém e em algum algoritmo que normlmente é especificado na nomenclatura do arquivo juntamente com o número de dimensões
word_vectors

## Operações com embeddings

A similaridade entre vetores de embeddings é dada pelo cosseno. Quanto mais próximo de 1 mais similar

In [ ]:
similarity = word_vectors.similarity('woman', 'man')
similarity

0.8323495

In [ ]:
word_vectors.similarity('vehicle', 'car')

0.86308384

In [ ]:
similarity = word_vectors.similarity('woman', 'fruit')
similarity

0.2577077

In [ ]:
import numpy as np
from numpy.linalg import norm

In [ ]:
A = word_vectors['woman']
B = word_vectors['man']

In [ ]:
# lembrando que o cosseno é o produto escalar normalizado
cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

Cosine Similarity: 0.83234936



queen = (king - man) + woman

In [ ]:
#testando o exemplo famoso do artigo do Mikolov sobre word2vec
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('queen', 0.7698540687561035)]


In [ ]:
result = word_vectors.most_similar(positive=['woman', 'programmer'], negative=['man'], topn=1)
print(result)

[('educator', 0.5853328704833984)]


## Material suplementar - outras embeddings

Glove: http://github.com/stanfordnlp/glove


Word2vec treinado no detalhe no Keras:
https://www.tensorflow.org/tutorials/text/word2vec


Doc2Vec

https://cs.stanford.edu/~quocle/paragraph_vector.pdf

https://alvinntnu.github.io/python-notes/nlp/doc2vec.html

# Exercícios para entregar

Carregue as embeddings indicadas em português para os três exercicios.

In [ ]:
#baixando as embeddings do NILC de http://nilc.icmc.usp.br/embeddings
!curl http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip > cbow_s100.zip
!unzip -o cbow_s100.zip
nomearq = 'cbow_s100.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  310M  100  310M    0     0  10.7M      0  0:00:28  0:00:28 --:--:-- 11.2M
Archive:  cbow_s100.zip
  inflating: cbow_s100.txt           


In [ ]:
%%time
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(nomearq, binary=False)
print('Carregado: ',nomearq)

Carregado:  cbow_s100.txt
CPU times: user 1min 12s, sys: 2.04 s, total: 1min 14s
Wall time: 1min 13s


In [ ]:
# total de palavras e as 10 primeiras
words = list(word_vectors.key_to_index)
print(f'O vocabulario contém {len(words)}')

O vocabulario contém 929606


## Exercício 1

A polissemia ocorre quando uma mesma palavra possui mais de um significado. Um exemplo de polissemia é a palavra “manga”, que pode ser parte de vestimenta ou uma fruta.

a) Usando a função `most_similar` do Gensim, analise o resultado para a palavra "manga". Escolha outra palavra polissêmica que exista no vocabulário e verifique as palavras mais similares.

In [ ]:
# seu codigo aqui

b) O que você observa e qual a sua hipótese para explicar esse comportamento.

> Sua análise aqui

## Exercício 2

a) Faça o exercício para duas palavras:

Escolha uma palavra, um sinônimo e um antônimo da mesma.
Calcule a distância euclideana e a similaridade do cosseno entre a palavra e seu sinônimo e a palavra e seu antônimo.






In [ ]:
# seu codigo aqui

b) O que você observa e qual a sua hipótese para explicar esse comportamento.

> Sua análise aqui

## Exercício 3

a) Verifique as palavras mais similares em relação às palavras "enfermeiro" e "enfermeira".

In [ ]:
# seu codigo aqui

b) Você observa algum viés? Se sim, qual sua hipótese.


> Sua análise aqui

